In [1]:
D.<x> = PolynomialRing(QQ)
R.<x> = PolynomialRing(QQ)
S = PowerSeriesRing(QQ,'x', default_prec=300)
L=LaurentSeriesRing(QQ,'x', default_prec=300)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=300)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.
def evaluate(expression,incomingvalue):
    expr=str(expression)
    output=sage_eval(expr,locals={'x':incomingvalue})
    return output

P=Primes()

def uniteList(lst):
    ln=len(lst)
    if ln<2:return lst
    answer=[]
    for j in [0..ln-1]:
        lj=lst[j]
        if (lj in answer)==False:
            answer=answer+[lj]
    return answer

def firsts(list):
    answer=[]
    for pair in list:
        answer=answer+[pair[0]]
    return answer

def prime(n): return P.unrank(n-1)

def unite(A,B):
    return set(A).union(B)

def factor_integer(n): #this is like "factor(integer)", but
    #the bases and exponents can be easily extracted.
    F=factor(n)
    return list(F)

def orderInteger(p,n):
    if n!=0:
        lst=n.digits(p)
        count=0
        k=0
        while lst[k]==0:
            count=count+1
            k=k+1
        return count
    if n==0:return(Infinity)

def order(p,fr):
    nn=numerator(QQ(fr))
    dn=denominator(QQ(fr))
    return orderInteger(p,nn)-orderInteger(p,dn)

def digitsum(p,n):return add(n.digits(p))

def digitsum(p,n):
    return sum(n.digits(p))

def rmnjntau(n):
    answer=0
    for k in [1..n-1]:
        answer=answer+(6*k-4*(n-k))*sigma(k,3)*sigma(n-k,5)
    answer=(5*sigma(n,3)+7*sigma(n,5))*n/12-35*answer
    return answer

def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])

def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer

def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))

def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))

def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)

def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))

def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)

def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)

def exNo3c(n,m):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # 
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.

def J(n,m):
    exn = exNo3c(n+2,m)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m):
    f = J(n,m)
    return f.truncate(n+1)

def Jcoefficient(n,m):
    f = J(n+2,m)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]

def xjNew(n,m):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(m**3*2**6)**i
        return rt
    return xjNewStep(n-1,m)

def j(n,m):return L(xjNew(n,m)/x)

def jpowerConstantTerm(m,power):
    trun=j(power,m)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def jLeoPowerConstantTerm(m,power,multiplier):
    trun=jLeo(power,m,multiplier)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def polynomialCoefficient(n,poly):
    return poly.list()[n]

def leadingCoefficient(polynomial):
    dg=polynomial.degree()
    
    return polynomialCoefficient(dg,polynomial)

def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent

def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)

def firstfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.first())

def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())

def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)

def element(k,listo):
    if k>len(listo):return (print("bad k"))
    return listo[k-1]

def mobiusInverseOfFunction(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        f=f+moebius(dvk)*g(n/dvk)
    return f

def mobiusInverseOfList(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        gndvk=element(n/dvk,g)
        f=f+moebius(dvk)*gndvk
    return f

def drop_first_element(listo):
    lenth=len(listo)
    answer=[]
    for k in [1..lenth-1]:
        answer=answer+[listo[k]]
    return answer

def factorSeriesFromLists(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
    return nswr

def factorSeriesFromListsWithTracking(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
            print((n,k))
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
        print("answer length: ",len(nswr))
    return nswr

def EulerMultiplication(listo):
    # Lemma 2.11 from my 1998 Experimental Mathematics paper.
    # Here the set A in the lemma is {1, 2, ...} = Z^+.
    p_A=[1] # so p_A[0]=1 as on top of p. 266, column 2.
    def ex(n):return listo[n-1] # ex is the exponent on the factor 1-x^n: -f(n)/n.
    def f(n):return -n*ex(n)
    answer=[1]
    def f_A(k):
        summ=0
        dk=divisors(k)
        for d in dk: #Here is where A = Z^+ is used to drop a constraint on k.
            summ=summ+f(d)
        return summ
    for n in [1..len(listo)-1]:
        sm=0
        for k in [1..n]:
            sm=sm+f_A(k)*p_A[n-k]
        p_A=p_A+[sm/n]
    return p_A

def listToPoly(listo):
    lenth=len(listo)
    answer=0
    for k in [1..lenth]:
        cf=listo[k-1]
        answer=answer+cf*x^(k-1)
    return answer

def difference_table(lst):
    diffs=[]
    for k in [0..len(lst)-2]:
        diff=lst[k+1]-lst[k]
        diffs=diffs+[diff]
    return diffs

def last_element(listo):return listo[len(listo)-1]
    
#Broadhurst code from #A005148:

#? {nt=100; a=[1]; b=[1]; d=1; e=0; g=0; print(1); for(n=2, nt, c=48*(a[n-1]+g)+128*(d-32*e); 
#e=d; d=c; i=(n-1)\2; g=12*if(n%2==0, a[n/2]^2)+24*sum(j=1, i, 
#a[j]*a[n-j]); h=12*if(n%2==0, b[n/2]^2)+24*sum(j=1, i, b[j]*b[n-j]); f=(c+5*h)/n^2-g; a=concat(a, f); 
#b=concat(b, n*f); print(f))}

#produces this on the PARI online evaluator (I have inserted the commas):
br=[
1,
47,
2488,
138799,
7976456,
467232200,
27736348480,
1662803271215,
100442427373480,
6103747246289272,
372725876150863808,
22852464771010647496,
1405886026610765892544,
86741060172969340021952,
5365190340823180439326208,
332577246704242939511725615,
20655377769544663820919905000,
1285027807539621869480480977880,
80066610886753513409821525593280,
4995543732566526565060187887772024,
312067903389730540416319245145039936,
19516459352109724206910675815791735872,
1221787478073080268912138739833447254528,
76558881238278398609546573647116818306504,
4801399849802188285872546222298724299377856,
301358552889212442951924121355286655092791360,
18928524108186605379268259069278244869735006720,
1189719542605042010945455887482239233732751142080,
74824958481405101799295401923145498080031496317440,
4708731584940969251488540213411242070133095720768000,
296483323638911778793802123013217365155428610625064960,
18677571039055424502042574350078071038555962934810664495,
1177200955467256907707767829606512556434525730284672082280,
74229820742983998523807878655148660941364964757170232076440,
4682657672641000613276353688819373189604961982881761635174080,
295516785862704112676947743865736338547152307208873658542187480,
18656838683258040776726836797753969443154060448210951169536087360,
1178287550937265649491805466460363896744099593833261406542090821440,
74441259433548426510664621182339422182178689134172479673100078686720,
4704546876230537649051669928635037299315044055233418643313504347890040,
297410696380227510473584821926459754598587577997951261584830786025989440,
18807176292551896455842616399574167855948518855982280636468413444438841280,
1189632505858785415664268185396568316810012962868095237190924015678644805120,
75269436592700558660145646818728077669744495747378078929068356710829357904960,
4763606735739477078702262301306618196904330454342036172567804617626114845601280,
301550219357655322958904198748139655940272138707157414253528789096123355242370560,
19093491105382437947961430595496009051927469794600124607374594862297809973497425920,
1209229421833128214532165231904398024088456532579184673374765702204525386892709582280,
76599462222171488217469562807555444840329820375936645628428503967599842536403748392640,
4853249476279584943018752544135518205835823652569328104071808597099976302206777672382272,
307557161322636240130429378891312616501367843861094990590342588813640658716687276702796288,
19494049310445477564550759598016449348244789590449694853790210431694353584090244058674665536,
1235828235382992966112000337356874542591602729215314383988292324949451319922660010288507026944,
78359386541981263753587663605771220395173695760309511724942887619218844604408365062990458445312,
4969331730256147172835336645727281480799885420753251882375600240456687485460922105379475213365248,
315192836223790555211163444188645825759965673786927108224693473418739558121668975477385609064230080,
19995098780164839843271975968833674908892528077762766665489021901583285029365260596621852077438476800,
1268636632413268672729488723673519658349225434504337043331170274695193536624267473803997222986457986560,
80503572319005599896519802085748511261953758070993177770252662999751870126880338970620619030455487016960,
5109226023515231795174271806642022238278119546486922652746346463140324125835625762209193286894097317188096,
324306079896787161344357124063601140357997183120543361083780244695916059297903737976316852087325736124084224,
20587951238469144728875413572545165973124146169862711202444824611729329948630917798996527215521428049827442688,
1307155886598940172607280751052393343723376964347599361863473016335357927316478471270092962849033836059872428032,
83003437890819298061159220674050513337514687509034267017635549288525596646068731914307086031798152640655708675631,
5271297646867701243231620056348828996858362139183014747261830107569871776680492045283621280463257824831450361228904,
334803625831811164273307471398151444570380979015726639675077730502474540099263374815508109958714263034979870045590040,
21267301347069209180434546119316800827596753601228462222166318226579132663526299067285136882052633679235269979706997440,
1351085301766957831759285167329838127692235945761375100182242807152550787271816136969058195839695684514251809964546971800,
85842015014782910888737358332857326909867676530325317899274801087388530640486646412863052257542933472850576480889534912320,
5454594181103314395353884318297045062026694474478123129221277721208935931363965378785241897523306877686001373372245607417152,
346632363267943630179319892957146981236491202095299908808662717047804243504180203803669349935923567992160546743779715198734848,
22030211252839896655760044436342222596667157343136792244243996326108725373610974640845764035532176284454550965374859141605876696,
1400264005045227498693320089227035884825680282492704474103652886750979292873550448366299419926933438094105780187281431855082851904,
89010607580108363146057947796447019984406819117601908324673767705214888723143847657440514237648023072092581303730690468541358446272,
5658653431087770147783431973072442739707731119479989465299403213540757386414610090412270459203277370032300491403324236240673058231808,
359768282283136451938233183643880156808956764689158714079901896700007719423392215459057530585195812985027222576505521483746145927094080,
22875473496936829687402292014161718834412927763836095316924893494178776078421448526507427925257601899589958056800447532066174066611688960,
1454634188002381701560543002374509285000151863499255133979293536524595484779236558931041033869616374153934389337102738325693541500885040640,
92506668310411752697700027500541107599054510647853447997767226456825933551220718898889759216482942932408592031625495444638507054727309701120,
5883380647402071727818124358464357357061997088366913641247755161122708769301392216445412497947130792332367814389247083022141893681918673035640,
374209367072642703061850255891790471856520862005092473938011251040764312378697398309875157737634899606364054132972840806767260748645213681887040,
23803199270417251468737963122626707467816366171461192749563400557478169477201165362512851772159448598935076348960799452562007578125691327998310080,
1514217230105491006416286735192399705713778112569360921940868647785308212475854144353099514116567802127049424293324092083010142613089652094239352320,
96332413045374746200370571514086365134691986056054938357641223739177015618938978964850794817403946266783867698429445521201166592631737075434674509760,
6128968041798698464871983060993177701445048464359248049771600533082817745158394358756109989641756654653420473533147391458535031474858708357818557350400,
389970917973786025826826148232023527188983145106526374860563378061488842725970203994300386046448714269543873579140545202535929354547230426525139471434240,
24814546341676335548842726357075976008312393230217108797033854347814848781823641548406242672165084256308341602099263161325277248861481206883551599766753280,
1579097865621621078127966349210038916312383695762720347829578839794195077696910123713050602263102661325782769915208166009373762048848807388370046579798291520,
100493898862353192639599085772479279507605308104393463045761183781449039477596456154787934584841158653853713852389459365917827655606822211804400142188630504960,
6395841086583513721518708585123859851483827511052356442692606484016955654747071476807355907729334535301712240745743084597082858081715137457733426771133436244480,
407082422122816769108928976084224767774083074925265772652509091404070682745260601092117551854989868576150378265801324011663291789547097044490460289150388722831360,
25911536639811395996751656925990326528992266850810482361355730044302538357241007798389242049209348961586470395969302758332164645822911593670231053397678686836037120,
1649413547559894591700394303117536943729317363777978884992676745553087694293098678096611231647446457954863017134436134803435906819926459313995976199288693719229255680,
105000403824773770333649105789820480034335517728840968988334310643726824123735041790377778857595055184234699203779438530862092851011368656739154398110404364696677847040,
6684622341158487811999668275612452018461145981041862416511208589860402818472403829783593014653257049064920715626593324371966426230798803277918448269747031588460412436480,
425585443813049916737778209891287760908166309712249413374261870396364033955750883326966977701620958916811619741413201896787531939944010422188516605008297556440177053390280,
27096933232548903092613115617140470313230857234611666991627504070866204307527763605986822544358886296299818124185350044201435781986714935884426869011696899042312957811757760,
1725347275200146778788065707502165945462707429554800950293313680035690477290460889039398857128345130378453865634762702538349329424603778897699271642610584658725955102258591040,
109864008941653383497254010859226498595190791378492674731068039950918263737791469856053624392850573136737164626778125500188292591996295723267255817207828065983109119673561029120,
6996107097499132971946127571962655485323611430800974100070703369809020527942862074490096967477739069312314066266444140870911945979977377211097174725520015644037734486845682825024]

def broadhurst(n):return br[n-1]

def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})

print("okay")

okay


In [ ]:
#output snipped; I interrupted this after k = 94.
polydata=[]
numericalfactordata=[]
broadhurst_values=[]
tests=[]
import time
for power in [1..100]:
    start=time.time()
    data=[]
    for m in [3..3*power+3]:
        cst= jpowerConstantTerm(m,power)
        data=data+[[m,cst]]
    rl=R.lagrange_polynomial(data)
    polydata=polydata+[[power,rl]]
    nf=ZZ(numericalfactor(rl))
    difference=nf/24-nszfunction(power)
    tests=tests+[difference]
    finish=time.time()
    print("(power,difference,time): ",(power,difference,finish-start))
print("===============================================================================")
print("tests:")
print(tests)
#below was originally done in a separate cell
#import pickle
#wfile = open('/Users/barrybrent/26oct22no1.txt','wb')
#t = pickle.dumps(str(polydata))
#wfile.write(t)
#wfile.close()

In [3]:
import pickle
rfile = open('/Users/barrybrent/26oct22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
p=5
for k in [1..1]: # k = 1 => power = 2
    data=[]
    power=s[k][0]
    print("----------------------------------------------------------------------------")
    print("power: ",power);print()
    poly=D(s[k][1]) # poly(m) = cst term of j_m^power
    nf=QQ(numericalfactor(poly))
    polyd=expand(poly/nf)
    column=[]
    for m in [3..503]:
        mp=ZZ(mod(m,p))
        mp2=ZZ(mod(m,p^2))
        dsm=digitsum(p,m)
        mpm=ZZ(mod(power,m))
        ex=order(p,m) 
        core=m/p^ex
        cmp=ZZ(mod(core,p))
        polydm=evaluate(polyd,m)
        o=order(p,polydm)
        zmo2=ZZ(mod(o,2))
        if  ex!=cmp-mp:data=data+[zmo2]
    print(data)

----------------------------------------------------------------------------
power:  2

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [8]:
import pickle
rfile = open('/Users/barrybrent/26oct22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
p=7
for k in [1..1]: # k = 1 => power = 2
    data=[]
    power=s[k][0]
    print("----------------------------------------------------------------------------")
    print("power: ",power);print()
    poly=D(s[k][1]) # poly(m) = cst term of j_m^power
    nf=QQ(numericalfactor(poly))
    polyd=expand(poly/nf)
    column=[]
    for m in [3..503]:
        mp=ZZ(mod(m,p))
        mp2=ZZ(mod(m,p^2))
        dsm=digitsum(p,m)
        mpm=ZZ(mod(power,m))
        ex=order(p,m) 
        core=m/p^ex
        cmp=ZZ(mod(core,p))
        polydm=evaluate(polyd,m)
        o=order(p,polydm)
        zmo2=ZZ(mod(o,2))
        if  ex!=cmp-mp:data=data+[zmo2]
    print(data)

----------------------------------------------------------------------------
power:  2

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [18]:
import pickle
rfile = open('/Users/barrybrent/26oct22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
for n in [3..30]:
    p=prime(n)
    data=[]
    for k in [1..1]: 
        power=s[k][0];print()
        print("power: ",power," prime: ",p)
        poly=D(s[k][1]) # poly(m) = cst term of j_m^power
        nf=QQ(numericalfactor(poly))
        polyd=expand(poly/nf)
        for m in [3..100*p]:
            mp=ZZ(mod(m,p))
            mp2=ZZ(mod(m,p^2))
            dsm=digitsum(p,m)
            mpm=ZZ(mod(power,m))
            ex=order(p,m) 
            core=m/p^ex
            cmp=ZZ(mod(core,p))
            polydm=evaluate(polyd,m)
            o=order(p,polydm)
            zmo2=ZZ(mod(o,2))
            if  ex!=cmp-mp:data=data+[zmo2]
    print(data)


power:  2  prime:  5
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

power:  2  prime:  7
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

power:  2  prime:  11
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

power:  2  prime:  13
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [19]:
import pickle
rfile = open('/Users/barrybrent/26oct22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
for n in [3..30]:
    p=prime(n)
    data=[]
    for k in [1..1]: 
        power=s[k][0];print()
        print("power: ",power," prime: ",p)
        poly=D(s[k][1]) # poly(m) = cst term of j_m^power
        nf=QQ(numericalfactor(poly))
        polyd=expand(poly/nf)
        for m in [3..100*p]:
            mp=ZZ(mod(m,p))
            mp2=ZZ(mod(m,p^2))
            dsm=digitsum(p,m)
            mpm=ZZ(mod(power,m))
            ex=order(p,m) 
            core=m/p^ex
            cmp=ZZ(mod(core,p))
            polydm=evaluate(polyd,m)
            o=order(p,polydm)
            zmo2=ZZ(mod(o,2))
            if  ex!=cmp-mp:data=data+[zmo2]
    print(data)


power:  2  prime:  5
[3, 3, 3, 0, 3, 3, 3, 3, 3, 3, 0, 3, 3, 3, 0, 3, 3, 3, 2, 3, 3, 3, 0, 3, 3, 3, 3, 3, 3, 0, 3, 3, 3, 0, 3, 3, 3, 2, 3, 3, 3, 0, 3, 3, 3, 3, 3, 3, 0, 3, 3, 3, 0, 3, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 3, 0, 3, 3, 3, 0, 3, 3, 3, 2]

power:  2  prime:  7
[2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 6, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 6, 2]

power:  2  prime:  11
[2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4]

power:  2  prime:  13
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]


In [20]:
import pickle
rfile = open('/Users/barrybrent/26oct22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
for n in [3..30]:
    p=prime(n)
    data=[]
    for k in [1..1]: 
        power=s[k][0];print()
        print("power: ",power," prime: ",p)
        poly=D(s[k][1]) # poly(m) = cst term of j_m^power
        nf=QQ(numericalfactor(poly))
        polyd=expand(poly/nf)
        for m in [3..100*p]:
            mp=ZZ(mod(m,p))
            mp2=ZZ(mod(m,p^2))
            dsm=digitsum(p,m)
            mpm=ZZ(mod(power,m))
            ex=order(p,m) 
            core=m/p^ex
            cmp=ZZ(mod(core,p))
            polydm=evaluate(polyd,m)
            o=order(p,polydm)
            zmo2=ZZ(mod(o,2))
            if  ex!=cmp-mp:data=data+[o]
    print(data)


power:  2  prime:  5
[3, 3, 3, 5, 3, 3, 3, 3, 3, 3, 5, 3, 3, 3, 5, 3, 3, 3, 7, 3, 3, 3, 5, 3, 3, 3, 3, 3, 3, 5, 3, 3, 3, 5, 3, 3, 3, 7, 3, 3, 3, 5, 3, 3, 3, 3, 3, 3, 5, 3, 3, 3, 5, 3, 3, 3, 3, 3, 3, 5, 3, 3, 3, 3, 3, 3, 5, 3, 3, 3, 5, 3, 3, 3, 7]

power:  2  prime:  7
[2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 6, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 6, 2]

power:  2  prime:  11
[2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4]

power:  2  prime:  13
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]


In [39]:
import pickle
rfile = open('/Users/barrybrent/26oct22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
for n in [3..30]:
    p=prime(n)
    data=[]
    for k in [1..1]: 
        power=s[k][0]
        poly=D(s[k][1]) # poly(m) = cst term of j_m^power
        nf=QQ(numericalfactor(poly))
        polyd=expand(poly/nf)
        for m in [3..100*p]:
            mp=ZZ(mod(m,p))
            mp2=ZZ(mod(m,p^2))
            dsm=digitsum(p,m)
            mpm=ZZ(mod(power,m))
            ex=order(p,m) 
            core=m/p^ex
            cmp=ZZ(mod(core,p))
            polydm=evaluate(polyd,m)
            o=order(p,polydm)
            mop=ZZ(mod(m,p))
            if  ex==cmp-mp:
                if o==0:
                    data=data+[mop]
    if data==[]:print("power: ",power," prime: ",p," empty")
    if data!=[]:
        ul=uniteList(data)
        print("power: ",power," prime: ",p," ul: ",ul)

power:  2  prime:  5  ul:  [3, 2]
power:  2  prime:  7  ul:  [3, 4, 5, 6, 1, 2]
power:  2  prime:  11  ul:  [3, 4, 5, 6, 7, 8, 9, 10, 1, 2]
power:  2  prime:  13  ul:  [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1, 2]
power:  2  prime:  17  ul:  [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 1, 2]
power:  2  prime:  19  ul:  [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 1, 2]
power:  2  prime:  23  ul:  [3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 19, 20, 21, 22, 1, 2]
power:  2  prime:  29  ul:  [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 1, 2]
power:  2  prime:  31  ul:  [3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 22, 23, 24, 25, 26, 27, 28, 29, 30, 1, 2]
power:  2  prime:  37  ul:  [3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 32, 33, 34, 35, 36, 1, 2]
power:  2  prime:  41  ul:  [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 

In [40]:
import pickle
rfile = open('/Users/barrybrent/26oct22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
for n in [3..30]:
    p=prime(n)
    data=[]
    for k in [1..1]: 
        power=s[k][0]
        poly=D(s[k][1]) # poly(m) = cst term of j_m^power
        nf=QQ(numericalfactor(poly))
        polyd=expand(poly/nf)
        for m in [3..100*p]:
            mp=ZZ(mod(m,p))
            mp2=ZZ(mod(m,p^2))
            dsm=digitsum(p,m)
            mpm=ZZ(mod(power,m))
            ex=order(p,m) 
            core=m/p^ex
            cmp=ZZ(mod(core,p))
            polydm=evaluate(polyd,m)
            o=order(p,polydm)
            mop=ZZ(mod(m,p))
            if o==0:data=data+[mop]
    if data==[]:print("power: ",power," prime: ",p," empty")
    if data!=[]:
        ul=uniteList(data)
        print("power: ",power," prime: ",p," ul: ",ul)

power:  2  prime:  5  ul:  [3, 2]
power:  2  prime:  7  ul:  [3, 4, 5, 6, 1, 2]
power:  2  prime:  11  ul:  [3, 4, 5, 6, 7, 8, 9, 10, 1, 2]
power:  2  prime:  13  ul:  [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1, 2]
power:  2  prime:  17  ul:  [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 1, 2]
power:  2  prime:  19  ul:  [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 1, 2]
power:  2  prime:  23  ul:  [3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 19, 20, 21, 22, 1, 2]
power:  2  prime:  29  ul:  [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 1, 2]
power:  2  prime:  31  ul:  [3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 22, 23, 24, 25, 26, 27, 28, 29, 30, 1, 2]
power:  2  prime:  37  ul:  [3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 32, 33, 34, 35, 36, 1, 2]
power:  2  prime:  41  ul:  [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 

In [28]:
import pickle
rfile = open('/Users/barrybrent/26oct22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
for n in [3..30]:
    p=prime(n)
    data=[]
    for k in [1..1]: 
        power=s[k][0]
        poly=D(s[k][1]) # poly(m) = cst term of j_m^power
        nf=QQ(numericalfactor(poly))
        polyd=expand(poly/nf)
        for m in [3..100*p]:
            mp=ZZ(mod(m,p))
            mp2=ZZ(mod(m,p^2))
            dsm=digitsum(p,m)
            mpm=ZZ(mod(power,m))
            ex=order(p,m) 
            core=m/p^ex
            cmp=ZZ(mod(core,p))
            polydm=evaluate(polyd,m)
            o=order(p,polydm)
            if  ex==cmp-mp:
                if o==1:
                    data=data+[m]
    if data!=[]:
        print("------------------------------------------------------------------------------")
        print("power: ",power," prime: ",p);print()
        print("data:");print(data);print();print("differences:")
        print(difference_table(data))

------------------------------------------------------------------------------
power:  2  prime:  5

data:
[4, 6, 9, 16, 19, 21, 24, 26, 29, 31, 34, 41, 44, 46, 49, 51, 54, 56, 59, 66, 69, 71, 74, 76, 79, 81, 84, 91, 94, 96, 99, 101, 104, 106, 109, 116, 119, 121, 124, 126, 129, 131, 134, 141, 144, 146, 149, 151, 154, 156, 159, 166, 169, 171, 174, 176, 179, 181, 184, 191, 194, 196, 199, 201, 204, 206, 209, 216, 219, 221, 224, 226, 229, 231, 234, 241, 244, 246, 249, 251, 254, 256, 259, 266, 269, 271, 274, 276, 279, 281, 284, 291, 294, 296, 299, 301, 304, 306, 309, 316, 319, 321, 324, 326, 329, 331, 334, 341, 344, 346, 349, 351, 354, 356, 359, 366, 369, 371, 374, 376, 379, 381, 384, 391, 394, 396, 399, 401, 404, 406, 409, 416, 419, 421, 424, 426, 429, 431, 434, 441, 444, 446, 449, 451, 454, 456, 459, 466, 469, 471, 474, 476, 479, 481, 484, 491, 494, 496, 499]

differences:
[2, 3, 7, 3, 2, 3, 2, 3, 2, 3, 7, 3, 2, 3, 2, 3, 2, 3, 7, 3, 2, 3, 2, 3, 2, 3, 7, 3, 2, 3, 2, 3, 2, 3, 7, 3, 2, 3, 2,

------------------------------------------------------------------------------
power:  2  prime:  71

data:
[14, 57, 85, 128, 156, 199, 227, 270, 298, 341, 369, 412, 440, 483, 511, 554, 582, 625, 653, 696, 724, 795, 838, 866, 909, 937, 980, 1008, 1051, 1079, 1122, 1150, 1193, 1221, 1264, 1292, 1335, 1363, 1406, 1434, 1477, 1505, 1548, 1576, 1619, 1647, 1690, 1718, 1761, 1789, 1832, 1860, 1903, 1931, 1974, 2002, 2045, 2073, 2116, 2144, 2187, 2215, 2258, 2286, 2329, 2357, 2400, 2428, 2471, 2499, 2542, 2570, 2613, 2641, 2684, 2712, 2755, 2783, 2826, 2854, 2897, 2925, 2968, 2996, 3039, 3067, 3110, 3138, 3181, 3209, 3252, 3280, 3323, 3351, 3394, 3422, 3465, 3493, 3536, 3564, 3607, 3635, 3678, 3706, 3749, 3777, 3820, 3848, 3891, 3919, 3962, 3990, 4033, 4061, 4104, 4132, 4175, 4203, 4246, 4317, 4345, 4388, 4416, 4459, 4487, 4530, 4558, 4601, 4629, 4672, 4700, 4743, 4771, 4814, 4842, 4885, 4913, 4956, 4984, 5027, 5055, 5098, 5126, 5169, 5197, 5240, 5268, 5311, 5339, 5382, 5410, 5453, 5481, 552

------------------------------------------------------------------------------
power:  2  prime:  113

data:
[43, 70, 156, 183, 269, 296, 382, 409, 495, 522, 608, 635, 721, 748, 834, 861, 947, 974, 1060, 1087, 1173, 1200, 1286, 1313, 1399, 1426, 1512, 1539, 1625, 1652, 1738, 1765, 1851, 1878, 1964, 1991, 2077, 2104, 2190, 2217, 2303, 2330, 2416, 2443, 2529, 2556, 2642, 2669, 2755, 2782, 2868, 2895, 2981, 3008, 3094, 3121, 3207, 3234, 3320, 3347, 3433, 3460, 3546, 3573, 3659, 3772, 3799, 3885, 3912, 3998, 4025, 4111, 4138, 4224, 4251, 4337, 4364, 4450, 4477, 4563, 4590, 4676, 4703, 4789, 4816, 4902, 4929, 5015, 5042, 5128, 5155, 5241, 5268, 5354, 5381, 5467, 5494, 5580, 5607, 5693, 5720, 5806, 5833, 5919, 5946, 6032, 6059, 6145, 6172, 6258, 6285, 6371, 6398, 6484, 6511, 6597, 6624, 6710, 6737, 6823, 6850, 6936, 6963, 7049, 7076, 7162, 7189, 7275, 7302, 7388, 7415, 7501, 7528, 7614, 7641, 7727, 7754, 7840, 7867, 7953, 7980, 8066, 8093, 8179, 8206, 8292, 8319, 8405, 8432, 8518, 8545, 8631

In [43]:
import pickle
rfile = open('/Users/barrybrent/26oct22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
for n in [3..30]:
    p=prime(n)
    data=[]
    for k in [1..1]: 
        power=s[k][0]
        poly=D(s[k][1]) # poly(m) = cst term of j_m^power
        nf=QQ(numericalfactor(poly))
        polyd=expand(poly/nf)
        for m in [3..100*p]:
            mp=ZZ(mod(m,p))
            mp2=ZZ(mod(m,p^2))
            dsm=digitsum(p,m)
            mpm=ZZ(mod(power,m))
            ex=order(p,m) 
            core=m/p^ex
            cmp=ZZ(mod(core,p))
            polydm=evaluate(polyd,m)
            o=order(p,polydm)
            if  ex!=cmp-mp:
                if o==0:
                    data=data+[m]
    if data!=[]: # no output
        print("------------------------------------------------------------------------------")
        print("power: ",power," prime: ",p);print()
        print("data:");print(data);print();print("differences:")
        print(difference_table(data))

In [42]:
import pickle
rfile = open('/Users/barrybrent/26oct22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
for n in [3..30]:
    p=prime(n)
    data=[]
    for k in [1..1]: 
        power=s[k][0]
        poly=D(s[k][1]) # poly(m) = cst term of j_m^power
        nf=QQ(numericalfactor(poly))
        polyd=expand(poly/nf)
        for m in [3..100*p]:
            mp=ZZ(mod(m,p))
            mp2=ZZ(mod(m,p^2))
            dsm=digitsum(p,m)
            mpm=ZZ(mod(power,m))
            ex=order(p,m) 
            core=m/p^ex
            cmp=ZZ(mod(core,p))
            polydm=evaluate(polyd,m)
            o=order(p,polydm)
            if  ex!=cmp-mp:
                if o==1:
                    data=data+[m]
    if data!=[]:
        print("------------------------------------------------------------------------------")
        print("power: ",power," prime: ",p);print()
        print("data:");print(data);print();print("differences:")
        print(difference_table(data))

------------------------------------------------------------------------------
power:  2  prime:  47

data:
[94, 141, 188, 235, 282, 329, 376, 423, 470, 517, 564, 611, 658, 705, 752, 799, 846, 893, 940, 987, 1034, 1081, 1128, 1175, 1222, 1269, 1316, 1363, 1410, 1457, 1504, 1551, 1598, 1645, 1692, 1739, 1786, 1833, 1880, 1927, 1974, 2021, 2068, 2115, 2162, 2303, 2350, 2397, 2444, 2491, 2538, 2585, 2632, 2679, 2726, 2773, 2820, 2867, 2914, 2961, 3008, 3055, 3102, 3149, 3196, 3243, 3290, 3337, 3384, 3431, 3478, 3525, 3572, 3619, 3666, 3713, 3760, 3807, 3854, 3901, 3948, 3995, 4042, 4089, 4136, 4183, 4230, 4277, 4324, 4371, 4512, 4559, 4606, 4653, 4700]

differences:
[47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 141, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47

In [41]:
import pickle
rfile = open('/Users/barrybrent/26oct22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
for n in [3..30]:
    p=prime(n)
    data=[]
    for k in [1..1]: 
        power=s[k][0]
        poly=D(s[k][1]) # poly(m) = cst term of j_m^power
        nf=QQ(numericalfactor(poly))
        polyd=expand(poly/nf)
        for m in [3..100*p]:
            mp=ZZ(mod(m,p))
            mp2=ZZ(mod(m,p^2))
            dsm=digitsum(p,m)
            mpm=ZZ(mod(power,m))
            ex=order(p,m) 
            core=m/p^ex
            cmp=ZZ(mod(core,p))
            polydm=evaluate(polyd,m)
            o=order(p,polydm)
            mop=ZZ(mod(m,p))
            if  ex==cmp-mp:
                if o==0:data=data+[mop]
    if data==[]:print("power: ",power," prime: ",p," empty")
    if data!=[]:
        ul=uniteList(data)
        print("power: ",power," prime: ",p," ul: ",ul)

power:  2  prime:  5  ul:  [3, 2]
power:  2  prime:  7  ul:  [3, 4, 5, 6, 1, 2]
power:  2  prime:  11  ul:  [3, 4, 5, 6, 7, 8, 9, 10, 1, 2]
power:  2  prime:  13  ul:  [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1, 2]
power:  2  prime:  17  ul:  [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 1, 2]
power:  2  prime:  19  ul:  [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 1, 2]
power:  2  prime:  23  ul:  [3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 19, 20, 21, 22, 1, 2]
power:  2  prime:  29  ul:  [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 1, 2]
power:  2  prime:  31  ul:  [3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 22, 23, 24, 25, 26, 27, 28, 29, 30, 1, 2]
power:  2  prime:  37  ul:  [3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 32, 33, 34, 35, 36, 1, 2]
power:  2  prime:  41  ul:  [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 

In [44]:
import pickle
rfile = open('/Users/barrybrent/26oct22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
for n in [3..30]:
    p=prime(n)
    data=[]
    for k in [1..1]: 
        power=s[k][0]
        poly=D(s[k][1]) # poly(m) = cst term of j_m^power
        nf=QQ(numericalfactor(poly))
        polyd=expand(poly/nf)
        for m in [3..100*p]:
            mp=ZZ(mod(m,p))
            mp2=ZZ(mod(m,p^2))
            dsm=digitsum(p,m)
            mpm=ZZ(mod(power,m))
            ex=order(p,m) 
            core=m/p^ex
            cmp=ZZ(mod(core,p))
            polydm=evaluate(polyd,m)
            o=order(p,polydm)
            mop=ZZ(mod(m,p))
            if  ex==cmp-mp:
                if o==1:data=data+[mop]
    if data==[]:print("power: ",power," prime: ",p," empty")
    if data!=[]:
        ul=uniteList(data)
        print("power: ",power," prime: ",p," ul: ",ul) 
        # solutions are \pm 1 modulo p, or zero. Satisfy a quadratic equation in Z/p?

power:  2  prime:  5  ul:  [4, 1]
power:  2  prime:  7  empty
power:  2  prime:  11  empty
power:  2  prime:  13  empty
power:  2  prime:  17  empty
power:  2  prime:  19  empty
power:  2  prime:  23  ul:  [5, 18]
power:  2  prime:  29  empty
power:  2  prime:  31  ul:  [10, 21]
power:  2  prime:  37  ul:  [6, 31]
power:  2  prime:  41  empty
power:  2  prime:  43  empty
power:  2  prime:  47  ul:  [0]
power:  2  prime:  53  empty
power:  2  prime:  59  ul:  [3, 56]
power:  2  prime:  61  empty
power:  2  prime:  67  empty
power:  2  prime:  71  ul:  [14, 57]
power:  2  prime:  73  empty
power:  2  prime:  79  empty
power:  2  prime:  83  empty
power:  2  prime:  89  ul:  [30, 59]
power:  2  prime:  97  ul:  [12, 85]
power:  2  prime:  101  empty
power:  2  prime:  103  ul:  [31, 72]
power:  2  prime:  107  empty
power:  2  prime:  109  empty
power:  2  prime:  113  ul:  [43, 70]


In [46]:
import pickle
rfile = open('/Users/barrybrent/26oct22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
for n in [3..30]:
    p=prime(n)
    data=[]
    for k in [1..1]: 
        power=s[k][0]
        poly=D(s[k][1]) # poly(m) = cst term of j_m^power
        nf=QQ(numericalfactor(poly))
        polyd=expand(poly/nf)
        for m in [3..100*p]:
            mp=ZZ(mod(m,p))
            mp2=ZZ(mod(m,p^2))
            dsm=digitsum(p,m)
            mpm=ZZ(mod(power,m))
            ex=order(p,m) 
            core=m/p^ex
            cmp=ZZ(mod(core,p))
            polydm=evaluate(polyd,m)
            o=order(p,polydm)
            mop=ZZ(mod(m,p))
            if o==0:data=data+[mop]
    if data==[]:print("power: ",power," prime: ",p," empty")
    if data!=[]:
        ul=uniteList(data)
        print("power: ",power," prime: ",p," ul: ",ul)

power:  2  prime:  5  ul:  [3, 2]
power:  2  prime:  7  ul:  [3, 4, 5, 6, 1, 2]
power:  2  prime:  11  ul:  [3, 4, 5, 6, 7, 8, 9, 10, 1, 2]
power:  2  prime:  13  ul:  [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1, 2]
power:  2  prime:  17  ul:  [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 1, 2]
power:  2  prime:  19  ul:  [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 1, 2]
power:  2  prime:  23  ul:  [3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 19, 20, 21, 22, 1, 2]
power:  2  prime:  29  ul:  [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 1, 2]
power:  2  prime:  31  ul:  [3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 22, 23, 24, 25, 26, 27, 28, 29, 30, 1, 2]
power:  2  prime:  37  ul:  [3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 32, 33, 34, 35, 36, 1, 2]
power:  2  prime:  41  ul:  [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 

In [45]:
import pickle
rfile = open('/Users/barrybrent/26oct22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
for n in [3..30]:
    p=prime(n)
    data=[]
    for k in [1..1]: 
        power=s[k][0]
        poly=D(s[k][1]) # poly(m) = cst term of j_m^power
        nf=QQ(numericalfactor(poly))
        polyd=expand(poly/nf)
        for m in [3..100*p]:
            mp=ZZ(mod(m,p))
            mp2=ZZ(mod(m,p^2))
            dsm=digitsum(p,m)
            mpm=ZZ(mod(power,m))
            ex=order(p,m) 
            core=m/p^ex
            cmp=ZZ(mod(core,p))
            polydm=evaluate(polyd,m)
            o=order(p,polydm)
            mop=ZZ(mod(m,p))
            if o==1:data=data+[mop]
    if data==[]:print("power: ",power," prime: ",p," empty")
    if data!=[]:
        ul=uniteList(data)
        print("power: ",power," prime: ",p," ul: ",ul) 
        # solutions are additive inverses modulo p: satisfy a quadratic equation in Z/p.

power:  2  prime:  5  ul:  [4, 1]
power:  2  prime:  7  empty
power:  2  prime:  11  empty
power:  2  prime:  13  empty
power:  2  prime:  17  empty
power:  2  prime:  19  empty
power:  2  prime:  23  ul:  [5, 18]
power:  2  prime:  29  empty
power:  2  prime:  31  ul:  [10, 21]
power:  2  prime:  37  ul:  [6, 31]
power:  2  prime:  41  empty
power:  2  prime:  43  empty
power:  2  prime:  47  ul:  [0]
power:  2  prime:  53  empty
power:  2  prime:  59  ul:  [3, 56]
power:  2  prime:  61  empty
power:  2  prime:  67  empty
power:  2  prime:  71  ul:  [14, 57]
power:  2  prime:  73  empty
power:  2  prime:  79  empty
power:  2  prime:  83  empty
power:  2  prime:  89  ul:  [30, 59]
power:  2  prime:  97  ul:  [12, 85]
power:  2  prime:  101  empty
power:  2  prime:  103  ul:  [31, 72]
power:  2  prime:  107  empty
power:  2  prime:  109  empty
power:  2  prime:  113  ul:  [43, 70]


In [47]:
import pickle
rfile = open('/Users/barrybrent/26oct22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
for n in [3..30]:
    p=prime(n)
    data=[]
    for k in [1..1]: 
        power=s[k][0]
        poly=D(s[k][1]) # poly(m) = cst term of j_m^power
        nf=QQ(numericalfactor(poly))
        polyd=expand(poly/nf)
        for m in [3..100*p]:
            mp=ZZ(mod(m,p))
            mp2=ZZ(mod(m,p^2))
            dsm=digitsum(p,m)
            mpm=ZZ(mod(power,m))
            ex=order(p,m) 
            core=m/p^ex
            cmp=ZZ(mod(core,p))
            polydm=evaluate(polyd,m)
            o=order(p,polydm)
            mop=ZZ(mod(m,p))
            if o==2:data=data+[mop]
    if data==[]:print("power: ",power," prime: ",p," empty")
    if data!=[]:
        ul=uniteList(data)
        print("power: ",power," prime: ",p," ul: ",ul) 

power:  2  prime:  5  ul:  [1, 4]
power:  2  prime:  7  ul:  [0]
power:  2  prime:  11  ul:  [0]
power:  2  prime:  13  ul:  [0]
power:  2  prime:  17  ul:  [0]
power:  2  prime:  19  ul:  [0]
power:  2  prime:  23  ul:  [0, 18, 5]
power:  2  prime:  29  ul:  [0]
power:  2  prime:  31  ul:  [0, 10, 21]
power:  2  prime:  37  ul:  [0, 31, 6]
power:  2  prime:  41  ul:  [0]
power:  2  prime:  43  ul:  [0]
power:  2  prime:  47  empty
power:  2  prime:  53  ul:  [0]
power:  2  prime:  59  ul:  [0, 56, 3]
power:  2  prime:  61  ul:  [0]
power:  2  prime:  67  ul:  [0]
power:  2  prime:  71  ul:  [0, 57, 14]
power:  2  prime:  73  ul:  [0]
power:  2  prime:  79  ul:  [0]
power:  2  prime:  83  ul:  [0]
power:  2  prime:  89  ul:  [0, 59, 30]
power:  2  prime:  97  ul:  [0, 85, 12]
power:  2  prime:  101  ul:  [0]
power:  2  prime:  103  ul:  [0, 31]
power:  2  prime:  107  ul:  [0]
power:  2  prime:  109  ul:  [0]
power:  2  prime:  113  ul:  [0, 70, 43]


In [49]:
import pickle
rfile = open('/Users/barrybrent/26oct22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
for n in [3..30]:
    p=prime(n)
    data=[]
    for k in [1..1]: 
        power=s[k][0]
        poly=D(s[k][1]) # poly(m) = cst term of j_m^power
        nf=QQ(numericalfactor(poly))
        polyd=expand(poly/nf)
        for m in [3..500*p]:
            mp=ZZ(mod(m,p))
            mp2=ZZ(mod(m,p^2))
            dsm=digitsum(p,m)
            mpm=ZZ(mod(power,m))
            ex=order(p,m) 
            core=m/p^ex
            cmp=ZZ(mod(core,p))
            polydm=evaluate(polyd,m)
            o=order(p,polydm)
            mop=ZZ(mod(m,p)) # = mp
            if o==3:data=data+[mop]
    if data==[]:print("power: ",power," prime: ",p," empty")
    if data!=[]:
        ul=uniteList(data)
        print("power: ",power," prime: ",p," ul: ",ul) 
# the fact that the same mod p solutions give o = 1,2,3, for example,
# says that another invariant determines o. Below, try cmp, mp2,  & m (mod p^ex).

power:  2  prime:  5  ul:  [0, 4, 1]
power:  2  prime:  7  empty
power:  2  prime:  11  empty
power:  2  prime:  13  empty
power:  2  prime:  17  empty
power:  2  prime:  19  empty
power:  2  prime:  23  ul:  [18, 5]
power:  2  prime:  29  empty
power:  2  prime:  31  ul:  [21]
power:  2  prime:  37  ul:  [31]
power:  2  prime:  41  empty
power:  2  prime:  43  empty
power:  2  prime:  47  ul:  [0]
power:  2  prime:  53  empty
power:  2  prime:  59  empty
power:  2  prime:  61  empty
power:  2  prime:  67  empty
power:  2  prime:  71  empty
power:  2  prime:  73  empty
power:  2  prime:  79  empty
power:  2  prime:  83  empty
power:  2  prime:  89  empty
power:  2  prime:  97  empty
power:  2  prime:  101  empty
power:  2  prime:  103  empty
power:  2  prime:  107  empty
power:  2  prime:  109  empty
power:  2  prime:  113  empty


In [3]:
import pickle
rfile = open('/Users/barrybrent/26oct22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
for n in [3..30]:
    p=prime(n)
    data=[]
    for k in [1..1]: 
        power=s[k][0]
        poly=D(s[k][1]) # poly(m) = cst term of j_m^power
        nf=QQ(numericalfactor(poly))
        polyd=expand(poly/nf)
        for m in [3..500*p]:
            mp=ZZ(mod(m,p))
            mp2=ZZ(mod(m,p^2))
            dsm=digitsum(p,m)
            mpm=ZZ(mod(power,m))
            ex=order(p,m) 
            core=m/p^ex
            cmp=ZZ(mod(core,p))
            polydm=evaluate(polyd,m)
            o=order(p,polydm)
            mop=ZZ(mod(m,p)) # = mp
            if o==0:data=data+[cmp]
    if data==[]:print("power: ",power," prime: ",p," empty")
    if data!=[]:
        ul=uniteList(data)
        print("power: ",power," prime: ",p," ul: ",ul) 

power:  2  prime:  5  ul:  [3, 2]
power:  2  prime:  7  ul:  [3, 4, 5, 6, 1, 2]
power:  2  prime:  11  ul:  [3, 4, 5, 6, 7, 8, 9, 10, 1, 2]
power:  2  prime:  13  ul:  [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1, 2]
power:  2  prime:  17  ul:  [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 1, 2]
power:  2  prime:  19  ul:  [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 1, 2]
power:  2  prime:  23  ul:  [3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 19, 20, 21, 22, 1, 2]
power:  2  prime:  29  ul:  [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 1, 2]
power:  2  prime:  31  ul:  [3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 22, 23, 24, 25, 26, 27, 28, 29, 30, 1, 2]
power:  2  prime:  37  ul:  [3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 32, 33, 34, 35, 36, 1, 2]
power:  2  prime:  41  ul:  [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 

In [4]:
import pickle
rfile = open('/Users/barrybrent/26oct22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
for n in [3..30]:
    p=prime(n)
    data=[]
    for k in [1..1]: 
        power=s[k][0]
        poly=D(s[k][1]) # poly(m) = cst term of j_m^power
        nf=QQ(numericalfactor(poly))
        polyd=expand(poly/nf)
        for m in [3..500*p]:
            mp=ZZ(mod(m,p))
            mp2=ZZ(mod(m,p^2))
            dsm=digitsum(p,m)
            mpm=ZZ(mod(power,m))
            ex=order(p,m) 
            core=m/p^ex
            cmp=ZZ(mod(core,p))
            polydm=evaluate(polyd,m)
            o=order(p,polydm)
            mop=ZZ(mod(m,p)) # = mp
            if o==1:data=data+[cmp]
    if data==[]:print("power: ",power," prime: ",p," empty")
    if data!=[]:
        ul=uniteList(data)
        print("power: ",power," prime: ",p," ul: ",ul) 

power:  2  prime:  5  ul:  [4, 1]
power:  2  prime:  7  empty
power:  2  prime:  11  empty
power:  2  prime:  13  empty
power:  2  prime:  17  empty
power:  2  prime:  19  empty
power:  2  prime:  23  ul:  [5, 18]
power:  2  prime:  29  empty
power:  2  prime:  31  ul:  [10, 21]
power:  2  prime:  37  ul:  [6, 31]
power:  2  prime:  41  empty
power:  2  prime:  43  empty
power:  2  prime:  47  ul:  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46]
power:  2  prime:  53  empty
power:  2  prime:  59  ul:  [3, 56]
power:  2  prime:  61  empty
power:  2  prime:  67  empty
power:  2  prime:  71  ul:  [14, 57]
power:  2  prime:  73  empty
power:  2  prime:  79  empty
power:  2  prime:  83  empty
power:  2  prime:  89  ul:  [30, 59]
power:  2  prime:  97  ul:  [12, 85]
power:  2  prime:  101  empty
power:  2  prime:  103  ul:  [31, 72]
power:  2  prime:  107  empty
pow

In [38]:
import pickle
rfile = open('/Users/barrybrent/26oct22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
for n in [3..30]:
    p=prime(n)
    data=[]
    data2=[]
    for k in [0..0]: 
        power=s[k][0]
        poly=D(s[k][1]) # poly(m) = cst term of j_m^power
        nf=QQ(numericalfactor(poly))
        polyd=expand(poly/nf)
        for m in [3..200]:
            if is_prime(m):
                ks=kronecker_symbol(p,m)
                polydm=evaluate(polyd,m)
                o=order(p,polydm)
                if ks==1:
                    diff=m-ZZ(mod(m,p))
                    if o==0:print((p,m,diff/p))

(5, 11, 2)
(5, 19, 3)
(5, 29, 5)
(5, 31, 6)
(5, 41, 8)
(5, 59, 11)
(5, 61, 12)
(5, 71, 14)
(5, 79, 15)
(5, 89, 17)
(5, 101, 20)
(5, 109, 21)
(5, 131, 26)
(5, 139, 27)
(5, 149, 29)
(5, 151, 30)
(5, 179, 35)
(5, 181, 36)
(5, 191, 38)
(5, 199, 39)
(7, 3, 0)
(7, 19, 2)
(7, 31, 4)
(7, 37, 5)
(7, 47, 6)
(7, 53, 7)
(7, 59, 8)
(7, 103, 14)
(7, 109, 15)
(7, 131, 18)
(7, 137, 19)
(7, 149, 21)
(7, 193, 27)
(7, 199, 28)
(11, 5, 0)
(11, 7, 0)
(11, 19, 1)
(11, 37, 3)
(11, 43, 3)
(11, 53, 4)
(11, 79, 7)
(11, 83, 7)
(11, 89, 8)
(11, 97, 8)
(11, 107, 9)
(11, 113, 10)
(11, 127, 11)
(11, 131, 11)
(11, 137, 12)
(11, 139, 12)
(11, 151, 13)
(11, 157, 14)
(11, 167, 15)
(11, 181, 16)
(13, 3, 0)
(13, 23, 1)
(13, 29, 2)
(13, 53, 4)
(13, 79, 6)
(13, 101, 7)
(13, 103, 7)
(13, 107, 8)
(13, 127, 9)
(13, 131, 10)
(13, 157, 12)
(13, 179, 13)
(13, 181, 13)
(17, 13, 0)
(17, 19, 1)
(17, 43, 2)
(17, 47, 2)
(17, 53, 3)
(17, 59, 3)
(17, 67, 3)
(17, 83, 4)
(17, 89, 5)
(17, 101, 5)
(17, 103, 6)
(17, 127, 7)
(17, 137, 8)
(17,

In [6]:
import pickle
rfile = open('/Users/barrybrent/26oct22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
for n in [3..30]:
    p=prime(n)
    data=[]
    data2=[]
    for k in [1..1]: 
        power=s[k][0]
        poly=D(s[k][1]) # poly(m) = cst term of j_m^power
        nf=QQ(numericalfactor(poly))
        polyd=expand(poly/nf)
        for m in [3..500*p]:
            mp=ZZ(mod(m,p))
            mp2=ZZ(mod(m,p^2))
            dsm=digitsum(p,m)
            mpm=ZZ(mod(power,m))
            ex=order(p,m) 
            core=m/p^ex
            cmp=ZZ(mod(core,p))
            polydm=evaluate(polyd,m)
            o=order(p,polydm)
            mop=ZZ(mod(m,p)) # = mp
            if o==0:data=data+[ex]
            if ex==0:data2=data2+[o]
        
        ul=uniteList(data)
        ul2=uniteList(data2)
        print("power: ",power," prime: ",p," ul: ",ul," ul2: ",ul2) 

power:  2  prime:  5  ul:  [0]  ul2:  [0, 1, 2, 3, 4, 5, 6]
power:  2  prime:  7  ul:  [0]  ul2:  [0]
power:  2  prime:  11  ul:  [0]  ul2:  [0]
power:  2  prime:  13  ul:  [0]  ul2:  [0]
power:  2  prime:  17  ul:  [0]  ul2:  [0]
power:  2  prime:  19  ul:  [0]  ul2:  [0]
power:  2  prime:  23  ul:  [0]  ul2:  [0, 1, 2, 3]
power:  2  prime:  29  ul:  [0]  ul2:  [0]
power:  2  prime:  31  ul:  [0]  ul2:  [0, 1, 2, 3]
power:  2  prime:  37  ul:  [0]  ul2:  [0, 1, 2, 3]
power:  2  prime:  41  ul:  [0]  ul2:  [0]
power:  2  prime:  43  ul:  [0]  ul2:  [0]
power:  2  prime:  47  ul:  []  ul2:  [-1]
power:  2  prime:  53  ul:  [0]  ul2:  [0]
power:  2  prime:  59  ul:  [0]  ul2:  [1, 0, 2]
power:  2  prime:  61  ul:  [0]  ul2:  [0]
power:  2  prime:  67  ul:  [0]  ul2:  [0]
power:  2  prime:  71  ul:  [0]  ul2:  [0, 1, 2]
power:  2  prime:  73  ul:  [0]  ul2:  [0]
power:  2  prime:  79  ul:  [0]  ul2:  [0]
power:  2  prime:  83  ul:  [0]  ul2:  [0]
power:  2  prime:  89  ul:  [0]  ul2:  [0,